In [18]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
import os
import weaviate
from weaviate.classes.init import Auth

wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)

In [19]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Settings.embed_model = embed_model

embeddings = embed_model.get_text_embedding(
    "OpenAI new Embeddings models is great."
)

print(embeddings[:5])

print("\n")

print(len(embeddings))

[-0.007731514051556587, 0.03945738077163696, -0.017062650993466377, -0.019995294511318207, 0.024583689868450165]


3072


In [20]:
# Ingest the docs in the data folder

docs = SimpleDirectoryReader('./data').load_data()

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(docs)

In [21]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore


# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Tesla")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)

In [16]:

query_engine = index.as_query_engine()
response = query_engine.query("reasons that my tesla's velocity is lower than usual")
print(response)

Your Tesla's velocity may be lower than usual due to a reduction in the vehicle's top speed and slower response to acceleration requests. Additionally, the maximum range of the vehicle may be reduced, and it may take longer to charge than before.


In [ ]:
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Tesla")
vector_store.delete_index()